```sql
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.
```

# **INTRODUCTIONS**


# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/conversions.ipynb",
        "src/utils/schemas.ipynb",
        "src/utils/protos.ipynb",
    ]
}
```

# **TABLES**

---


### STAGINGS

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTAGE].[CIMS_VACCINATED_AGE_GROUP_CAMPAIGN]'))
CREATE TABLE [VWSSTAGE].[CIMS_VACCINATED_AGE_GROUP_CAMPAIGN](
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
  [VERSION] [VARCHAR](100) NULL,
  [DATE_OF_REPORT] [VARCHAR](100) NULL,
  [DATE_OF_STATISTICS_WEEK_START] [VARCHAR](100) NULL,
	[DATE_OF_STATISTICS_WEEK_END] [VARCHAR](100) NULL,
	[BIRTH_COHORT] [VARCHAR](100) NULL,
  [AGE_GROUP] [VARCHAR](100) NULL,
  [POPULATION_AGE_GROUP] [VARCHAR](100) NULL,
  [VACCINE_SERIE] [VARCHAR](100) NULL,
  [RECEIVED] [VARCHAR](100) NULL,
  [COVERAGE] [VARCHAR](100) NULL,
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE()
);
GO

### INTERMEDIATES

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSINTER].[CIMS_VACCINATED_AGE_GROUP_CAMPAIGN]'))
CREATE TABLE [VWSINTER].[CIMS_VACCINATED_AGE_GROUP_CAMPAIGN](
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
  [DATE_OF_REPORT] [DATETIME] NULL,
  [DATE_OF_STATISTICS_WEEK_START] [DATETIME] NULL,
	[DATE_OF_STATISTICS_WEEK_END] [DATETIME] NULL,
	[BIRTH_COHORT] [VARCHAR](100) NULL,
  [AGE_GROUP] [VARCHAR](100) NULL,
  [POPULATION_AGE_GROUP] [BIGINT] NULL,
  [VACCINE_SERIE] [VARCHAR](100) NULL,
  [RECEIVED] [BIGINT] NULL,
  [COVERAGE] [DECIMAL](16,2) NULL,
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE()
);
GO

### DESTINATIONS

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[CIMS_VACCINATED_AGE_GROUP_CAMPAIGN]'))
CREATE TABLE [VWSDEST].[CIMS_VACCINATED_AGE_GROUP_CAMPAIGN](
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
  [DATE_OF_REPORT] DATETIME NULL,
  [DATE_OF_STATISTICS_WEEK_START] DATETIME NULL,
  [DATE_OF_STATISTICS_WEEK_END] DATETIME NULL,
  [BIRTH_COHORT] VARCHAR(100) NULL,
  [AGE_GROUP] VARCHAR(100) NULL,
  [AGE_GROUP_POPULATION] BIGINT NULL,
  [AGE_GROUP_POPULATION_PERCENTAGE] DECIMAL(16,2) NULL,
  [VACCINE_SERIE] [VARCHAR](100) NULL,
  [RECEIVED] [BIGINT] NULL,
  [COVERAGE] [DECIMAL](16,2) NULL,
  [DATE_LAST_INSERTED] DATETIME DEFAULT GETDATE()
);
GO

-- 2) CREATE INDEX(ES).....
DROP INDEX IF EXISTS [NCIX_DLI_DOS_AG_CIMS_VACCINATED_AGE_GROUP_CAMPAIGN] 
  ON [VWSDEST].[CIMS_VACCINATED_AGE_GROUP_CAMPAIGN]
GO

CREATE NONCLUSTERED INDEX [NCIX_DLI_DOS_AG_CIMS_VACCINATED_AGE_GROUP_CAMPAIGN] 
  ON [VWSDEST].[CIMS_VACCINATED_AGE_GROUP_CAMPAIGN] (
    [DATE_LAST_INSERTED], 
    [DATE_OF_STATISTICS_WEEK_START], 
    [DATE_OF_STATISTICS_WEEK_END], 
    [AGE_GROUP]
  );
GO

# **STORED PROCEDURES**

---


### STAGING &rarr; INTERMEDIATE MAPPING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT. 
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE OR ALTER PROCEDURE [DBO].[SP_INTER_CIMS_VACCINATED_AGE_GROUP_CAMPAIGN]
AS
BEGIN
    INSERT INTO [VWSINTER].[CIMS_VACCINATED_AGE_GROUP_CAMPAIGN] (
        [DATE_OF_REPORT],
        [DATE_OF_STATISTICS_WEEK_START],
        [DATE_OF_STATISTICS_WEEK_END],
        [BIRTH_COHORT],
        [AGE_GROUP],
        [POPULATION_AGE_GROUP],
        [VACCINE_SERIE],
        [RECEIVED],
        [COVERAGE],
    )
    SELECT
        [DATE_OF_REPORT],
        [DATE_OF_STATISTICS_WEEK_START],
        [DATE_OF_STATISTICS_WEEK_END],
        [BIRTH_COHORT],
        [AGE_GROUP],
        [POPULATION_AGE_GROUP],
        [VACCINE_SERIE],
        [RECEIVED],
        [COVERAGE]
    FROM [VWSSTAGE].[CIMS_VACCINATED_AGE_GROUP_CAMPAIGN]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[CIMS_VACCINATED_AGE_GROUP_CAMPAIGN]);
END;
GO

### INTERMEDIATE &rarr; DESTINATION MAPPING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE OR ALTER  PROCEDURE [DBO].[SP_DEST_CIMS_VACCINATED_AGE_GROUP_CAMPAIGN]
AS
BEGIN
-- APPLY FILTERS ON MAIN INPUT TABLE AND PERFORM TRANSFORMATIONS
WITH LAST_CTE AS (
    SELECT
        [DATE_OF_REPORT],
        [DATE_OF_STATISTICS_WEEK_START],
        [DATE_OF_STATISTICS_WEEK_END],
        [BIRTH_COHORT],
        [AGE_GROUP],
        [POPULATION_AGE_GROUP],
        [VACCINE_SERIE],
        [RECEIVED],
        [COVERAGE]
    FROM [VWSINTER].[CIMS_VACCINATED_AGE_GROUP_CAMPAIGN]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSINTER].[CIMS_VACCINATED_AGE_GROUP_CAMPAIGN])
        AND LOWER([BIRTH_COHORT]) != 'unknown'
)
INSERT INTO [VWSDEST].[CIMS_VACCINATED_AGE_GROUP_CAMPAIGN](
    [DATE_OF_REPORT],
    [DATE_OF_STATISTICS_WEEK_START],
    [DATE_OF_STATISTICS_WEEK_END],
    [AGE_GROUP],
    [VACCINE_SERIE],
    [RECEIVED],
    [COVERAGE]
)
SELECT
    T.[DATE_OF_REPORT],
    T.[DATE_OF_STATISTICS_WEEK_START],
    T.[DATE_OF_STATISTICS_WEEK_END],
    T.[AGE_GROUP],
    T.[VACCINE_SERIE],
    T.[RECEIVED],
    T.[COVERAGE]
    FROM LAST_CTE AS T
    --TAKE ONLY THE LAST VALUE
    WHERE T.[DATE_OF_STATISTICS_WEEK_START] = (SELECT MAX([DATE_OF_STATISTICS_WEEK_START]) FROM LAST_CTE)
END
GO

# **ORCHESTRATOR CONFIGURATIONS**

---

> WORKFLOWS

In [ ]:
-- 1) UPSERT WORKFLOW(S).....
DECLARE @workflow_name NVARCHAR(50) = 'CIMS_VACCINATED_PER_AGE_GROUP',
        @workflow_id INT,        
        @workflow_description VARCHAR(256),
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 0
    WHEN 'acceptance' THEN 0
    ELSE 0
END;

SELECT TOP(1)
    @workflow_id = workflows.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] AND v_workflows.[ID] = workflows.[ID]
WHERE v_workflows.[NAME] = @workflow_name;

SET @workflow_description = CONCAT('WORKFLOW: ', @workflow_name);

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
    @id = @workflow_id, 
    @workflow_name = @workflow_name,
    @description = @workflow_description,
    @schedule = '0 12 * * WED', -- AT 02:00 PM, ON WEDNESDAY
    @active = @is_active;
GO

> PROTOS

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_RIVM_CIMS_VACCINATED_PER_AGE_GROUP',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'vaccine_coverage_per_age_group',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 2, -- VALUES
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;
    
SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_RIVM_CIMS_VACCINATED_PER_AGE_GROUP_ARCHIVE',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'vaccine_coverage_per_age_group_archived',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 2, -- VALUES
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;
    
SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO